# SETUP

## Import Libraries

In [6]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

## Read MongoDB

In [7]:
client = MongoClient('mongodb://localhost:27017')
db = client["test"]

## Initialize Output DataFrame

In [8]:
output_df = pd.DataFrame()

# MATCHAPPS

## Matchapps Initalization

### Create Matchapps DataFrame

In [9]:
matchapps_collection = db["matchapps"]
matchapps_df = matchapps_collection.find()

### Flatten, reorganize, and clean Data

In [10]:
matchapps_df = pd.json_normalize(matchapps_df)

matchapps_column_order = [
    "_id",
    "metadata.matchNumber",
    "metadata.scouterName",
    "metadata.robotTeam",
    "metadata.robotPosition",
    "leftStartingZone",
    "climb",
    "trapNotes",
    "autoNotes.near",
    "autoNotes.mid",
    "autoNotes.far",
    "autoNotes.amp",
    "autoNotes.miss",
    "teleNotes.near",
    "teleNotes.mid",
    "teleNotes.far",
    "teleNotes.amp",
    "teleNotes.miss"   
]

matchapps_df = matchapps_df[matchapps_column_order]
matchapps_df.sort_values("metadata.matchNumber", inplace=True)
matchapps_df

,_id,metadata.matchNumber,metadata.scouterName,metadata.robotTeam,metadata.robotPosition,leftStartingZone,climb,trapNotes,autoNotes.near,autoNotes.mid,autoNotes.far,autoNotes.amp,autoNotes.miss,teleNotes.near,teleNotes.mid,teleNotes.far,teleNotes.amp,teleNotes.miss
0,65f29f03799a6643610013e5,1,Jim,9642,red_1,False,park,0,2,2,2,2,0,0,2,2,2,4
1,65f29f03799a6643610013e9,1,Jim,4723,red_2,False,center,1,0,1,0,3,4,4,3,8,1,5
2,65f29f03799a6643610013ed,1,Jim,7571,red_3,True,park,0,0,2,3,4,1,2,2,9,5,8
3,65f29f03799a6643610013f1,1,Jim,3682,blue_1,True,center,1,3,1,2,4,0,5,6,4,3,2
4,65f29f03799a6643610013f5,1,Jim,428,blue_2,False,none,1,0,0,4,3,4,3,5,8,9,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390,65f29f0e799a66436100393d,399,Jim,894,red_3,True,park,0,1,3,2,4,0,2,4,6,8,3
2392,65f29f0e799a664361003945,399,Jim,2555,blue_2,False,none,1,4,1,4,4,3,7,6,3,8,0
2388,65f29f0e799a664361003935,399,Jim,46,red_1,False,none,1,4,2,4,1,1,2,0,4,9,4
2389,65f29f0e799a664361003939,399,Jim,4025,red_2,True,center,0,4,2,4,4,2,4,8,6,5,4


## Matchapps Function and Variable Initializations

### Check Alliance Harmonization Function Definition

In [11]:
def check_team_harmonization_in_alliance(t1, t2, t3):
    t1_status, t2_status, t3_status = False, False, False
    if t1 not in ["failed", "none", "park"]:
        if t1 == t2:
            t1_status = True
            t2_status = True
        if t1 == t3:
            t1_status = True
            t3_status = True
    elif t2 not in ["failed", "none", "park"]:
        if t2 == t3:
            t2_status = True
            t3_status = True
    return t1_status, t2_status, t3_status

### Complete List of all Teams in DF

In [12]:
team_list = matchapps_df["metadata.robotTeam"].unique().tolist()

### Complete Range of All Matches

In [13]:
match_range = matchapps_df["metadata.matchNumber"].max()

## Matchapps Data Analysis

### Auto Data Analysis Calculations

#### Auto Speaker Notes Average (near, mid, far)

In [14]:
output_df["Auto Speaker Notes Near Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.near"].mean()
output_df["Auto Speaker Notes Mid Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.mid"].mean()
output_df["Auto Speaker Notes Far Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.far"].mean()

output_df["Auto Speaker Notes Total Avg"] = None

for team in team_list:
    output_df.at[team, "Auto Speaker Notes Total Avg"] = sum([output_df.at[team, "Auto Speaker Notes Near Avg"], output_df.at[team, "Auto Speaker Notes Mid Avg"], output_df.at[team, "Auto Speaker Notes Far Avg"]])

In [15]:

grouped = matchapps_df.groupby("metadata.robotTeam")
output_df["Auto Speaker Notes Near Max"] = grouped["autoNotes.near"].max()
output_df["Auto Speaker Notes Mid Max"] = grouped["autoNotes.mid"].max()
output_df["Auto Speaker Notes Far Max"] = grouped["autoNotes.far"].max()
output_df["Auto Speaker Notes Total Max"] = (matchapps_df[["autoNotes.far", "autoNotes.mid", "autoNotes.near"]].sum(axis=1)).groupby(matchapps_df['metadata.robotTeam']).max()

output_df

,Auto Speaker Notes Near Avg,Auto Speaker Notes Mid Avg,Auto Speaker Notes Far Avg,Auto Speaker Notes Total Avg,Auto Speaker Notes Near Max,Auto Speaker Notes Mid Max,Auto Speaker Notes Far Max,Auto Speaker Notes Total Max
metadata.robotTeam,,,,,,,,
11,1.947368,2.052632,2.421053,6.421053,4,4,4,10
46,1.724138,2.068966,2.379310,6.172414,4,4,4,12
62,2.111111,1.740741,2.222222,6.074074,4,4,4,11
178,1.055556,2.666667,2.333333,6.055556,3,4,4,10
428,1.800000,2.160000,1.680000,5.64,4,4,4,10
...,...,...,...,...,...,...,...,...
9577,1.640000,2.400000,2.120000,6.16,4,4,4,10
9642,1.476190,1.666667,2.476190,5.619048,4,4,4,9
9702,1.714286,2.095238,1.476190,5.285714,4,4,4,10


#### Auto Amp Notes Average

In [16]:
output_df["Auto Notes Amp Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.amp"].mean()

#### Auto Percent of Miss Notes

In [17]:
output_df["Auto Percent of Missed Notes"] = None

 # CHANGE FORMAT LATER
output_df["Auto Max Missed Notes"] = None
for team in team_list:
    missed_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.miss"].sum()
    total_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.near"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.mid"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.far"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.amp"].sum()
    + missed_notes
    output_df.at[team, "Auto Percent of Missed Notes"] = missed_notes / total_notes
    
     # CHANGE FORMAT LATER

    output_df.at[team, "Auto Max Missed Notes"] = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.amp"].max()

#### Auto All Notes Average

In [18]:
output_df["Auto Notes Avg"] = (output_df["Auto Notes Amp Avg"] +
                               output_df["Auto Speaker Notes Near Avg"] +
                               output_df["Auto Speaker Notes Mid Avg"] +
                               output_df["Auto Speaker Notes Far Avg"]) / 4

#### Average Auto points

In [19]:
output_df['Auto Points Avg'] = (5 * (matchapps_df['teleNotes.near'] + matchapps_df['teleNotes.mid'] + matchapps_df['teleNotes.far']) + 2 * matchapps_df['teleNotes.amp']).groupby(matchapps_df['metadata.robotTeam']).mean()

### Teleop Data Analysis Calculations

#### Teleop Speaker Notes Average (near, mid, far)

In [20]:
output_df["Tele Speaker Notes Near Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.near"].mean()
output_df["Tele Speaker Notes Mid Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.mid"].mean()
output_df["Tele Speaker Notes Far Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.far"].mean()

output_df["Tele Speaker Notes Total Avg"] = None

for team in team_list:
    output_df.at[team, "Tele Speaker Notes Total Avg"] = sum([output_df.at[team, "Tele Speaker Notes Near Avg"], output_df.at[team, "Tele Speaker Notes Mid Avg"], output_df.at[team, "Tele Speaker Notes Far Avg"]])

In [21]:
output_df["Tele Speaker Notes Near Max"] = grouped["teleNotes.near"].max()
output_df["Tele Speaker Notes Mid Max"] = grouped["teleNotes.mid"].max()
output_df["Tele Speaker Notes Far Max"] = grouped["teleNotes.far"].max()
output_df["Tele Speaker Notes Total Max"] = (matchapps_df[["teleNotes.far", "teleNotes.mid", "teleNotes.near"]].sum(axis=1)).groupby(matchapps_df['metadata.robotTeam']).max()

output_df

,Auto Speaker Notes Near Avg,Auto Speaker Notes Mid Avg,Auto Speaker Notes Far Avg,Auto Speaker Notes Total Avg,Auto Speaker Notes Near Max,Auto Speaker Notes Mid Max,Auto Speaker Notes Far Max,Auto Speaker Notes Total Max,Auto Notes Amp Avg,Auto Percent of Missed Notes,...,Auto Notes Avg,Auto Points Avg,Tele Speaker Notes Near Avg,Tele Speaker Notes Mid Avg,Tele Speaker Notes Far Avg,Tele Speaker Notes Total Avg,Tele Speaker Notes Near Max,Tele Speaker Notes Mid Max,Tele Speaker Notes Far Max,Tele Speaker Notes Total Max
metadata.robotTeam,,,,,,,,,,,,,,,,,,,,,
11,1.947368,2.052632,2.421053,6.421053,4,4,4,10,2.631579,1.054054,...,2.263158,69.526316,3.947368,4.210526,4.105263,12.263158,9,9,8,22
46,1.724138,2.068966,2.379310,6.172414,4,4,4,12,1.965517,1.26,...,2.034483,74.793103,3.862069,4.448276,4.896552,13.206897,9,9,9,23
62,2.111111,1.740741,2.222222,6.074074,4,4,4,11,2.111111,0.789474,...,2.046296,77.555556,4.370370,4.444444,4.814815,13.62963,9,9,9,24
178,1.055556,2.666667,2.333333,6.055556,3,4,4,10,1.944444,2.263158,...,2.000000,74.777778,4.111111,4.555556,4.666667,13.333333,9,9,9,24
428,1.800000,2.160000,1.680000,5.64,4,4,4,10,1.720000,1.311111,...,1.840000,78.120000,3.960000,5.320000,4.520000,13.8,8,9,9,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9577,1.640000,2.400000,2.120000,6.16,4,4,4,10,2.160000,1.317073,...,2.080000,77.880000,4.320000,4.840000,4.480000,13.64,9,9,9,26
9642,1.476190,1.666667,2.476190,5.619048,4,4,4,9,1.476190,1.290323,...,1.773810,64.714286,4.285714,3.857143,3.428571,11.571429,9,9,9,25
9702,1.714286,2.095238,1.476190,5.285714,4,4,4,10,2.333333,1.083333,...,1.904762,74.095238,4.428571,4.095238,4.238095,12.761905,8,9,9,20


#### Teleop Amp Notes Average

In [22]:
output_df["Tele Notes Amp Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.amp"].mean()

 # CHANGE FORMAT LATER
output_df["Tele Max Amp Notes"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.amp"].max()

#### Teleop All Notes Average

In [23]:
output_df["Tele Notes Avg"] = (output_df["Tele Notes Amp Avg"] +
                               output_df["Tele Speaker Notes Near Avg"] +
                               output_df["Tele Speaker Notes Mid Avg"] +
                               output_df["Tele Speaker Notes Far Avg"]) / 4

 # CHANGE FORMAT LATER
output_df["Tele Max Notes"] = None

#### Teleop Percent of Missed Notes

In [24]:
output_df["Tele Percent of Missed Notes"] = None

 # CHANGE FORMAT LATER
output_df["Tele Max Missed Notes"] = None
for team in team_list:
    missed_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.miss"].sum()
    total_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.near"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.mid"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.far"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.amp"].sum()
    + missed_notes
    output_df.at[team, "Percent of Missed Notes"] = missed_notes / total_notes
    
     # CHANGE FORMAT LATER

    output_df.at[team, "Tele Max Missed Notes"] = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.amp"].max()

In [25]:
output_df["Tele Speaker Notes Near Max"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.near"].max()
output_df["Tele Speaker Notes Mid Max"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.mid"].max()
output_df["Tele Speaker Notes Far Max"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.far"].max()

#### Average Teleop Points

In [26]:
output_df['Tele Points Avg'] = (2 * (matchapps_df['teleNotes.near'] + matchapps_df['teleNotes.mid'] + matchapps_df['teleNotes.far']) + 1 * matchapps_df['teleNotes.amp']).groupby(matchapps_df['metadata.robotTeam']).mean()

In [27]:
# output_df['Endgame Points Avg'] =
#(np.where(matchapps_df['trapNotes']), 0, 1)
np.where(matchapps_df['climb'].isin(['source', 'center', 'amp']), 0, 3)


array([3, 0, 3, ..., 3, 0, 0])

### Other Data Analysis Calculations

#### Trap Notes

In [28]:
output_df["Trap Notes Avg"] = matchapps_df.groupby("metadata.robotTeam")["trapNotes"].mean()

#### Percent of Successful Climbs and Successful Climbs Total

In [29]:
output_df["Percent of Successful Climbs"] = None
output_df["Successful Climbs Total"] = None

for team in team_list:
    climb_list = matchapps_df.loc[matchapps_df["metadata.robotTeam"] == team, "climb"].tolist()
    total_climbs = 0
    successful_climbs = 0
    for climb_type in climb_list:
        if climb_type == "failed":
            total_climbs += 1
        elif climb_type == "amp" or climb_type == "source" or climb_type == "center":
            total_climbs += 1
            successful_climbs += 1
    if total_climbs == 0:
        output_df.at[team, "Percent of Successful Climbs"] = 0
        
    else:
        output_df.at[team, "Percent of Successful Climbs"] = successful_climbs / total_climbs
    
    output_df.at[team, "Successful Climbs Total"] = successful_climbs

#### Harmonized Climbs Total

#### Percent of Harmonized Climbs out of Successful Climbs

In [30]:
# output_df["Percent of Harmonized Climbs out of Successful Climbs"] = 0

# output_df.index

# for team in team_list:
#     output_df.at[team, "Percent of Harmonized Climbs out of Successful Climbs"] = output_df.at[team, "Harmonized Climbs Total"] / output_df.at[team, "Successful Climbs Total"]

#### Percent of Times Robot Left Starting Zone During Autos

In [31]:
output_df["Percent of Times Robot Left Starting Zone During Autos"] = matchapps_df.groupby("metadata.robotTeam")["leftStartingZone"].mean()

### Custom Metric Data Analysis Calculations

#### Endgame Custom Metric

In [32]:
# output_df["Endgame Custom Metric"] = ((output_df["Percent of Successful Climbs"] * 5) ** 2 + (output_df["Percent of Harmonized Climbs out of Successful Climbs"] * 5) ** 2 + (output_df["Trap Notes Avg"] * 5) ** 2) / 3
# output_df["Endgame Custom Metric"] = output_df["Endgame Custom Metric"].apply(lambda x: round(x, 1))

# SUPERAPPS

## Supperapps Initalization

### Create Superapps DataFrame

In [33]:
superapps_collection = db["superapps"]
superapps_df = superapps_collection.find()

### Flatten, reorganize, and clean Data

In [34]:
superapps_df = pd.json_normalize(superapps_df)
superapps_df.sort_values("metadata.matchNumber", inplace=True)
superapps_df

,_id,defense,defended,comments,__v,metadata.scouterName,metadata.matchNumber,metadata.robotTeam,metadata.robotPosition,fouls.insideRobot,fouls.protectedZone,fouls.pinning,fouls.multiplePieces,fouls.other,humanShooter.highNotes.amp,humanShooter.highNotes.source,humanShooter.highNotes.center
0,65f29f03799a6643610013e7,someDef,False,"[source_only, avoids_under_stage]",0,Jim,1,9642,red_1,0,1,0,0,1,True,True,False
1,65f29f03799a6643610013eb,someDef,False,"[good_driving, source_only, clogging, sturdy_b...",0,Jim,1,4723,red_2,0,1,1,1,0,NaN,NaN,NaN
2,65f29f03799a6643610013ef,someDef,True,"[great_driving, clogging, effective_defense, m...",0,Jim,1,7571,red_3,1,0,0,0,0,NaN,NaN,NaN
3,65f29f03799a6643610013f3,someDef,False,"[clogging, mid_defense, sturdy_build]",0,Jim,1,3682,blue_1,1,0,1,1,0,True,True,True
4,65f29f03799a6643610013f7,noDef,False,"[good_driving, source_only, avoids_under_stage]",0,Jim,1,428,blue_2,0,0,0,1,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390,65f29f0e799a66436100393f,noDef,False,"[great_driving, sturdy_build]",0,Jim,399,894,red_3,1,0,0,1,1,False,False,True
2392,65f29f0e799a664361003947,noDef,False,"[clogging, effective_defense]",0,Jim,399,2555,blue_2,1,1,1,1,1,NaN,NaN,NaN
2388,65f29f0e799a664361003937,noDef,True,"[great_driving, ineffective_defense, sturdy_bu...",0,Jim,399,46,red_1,0,0,1,1,1,NaN,NaN,NaN
2389,65f29f0e799a66436100393b,someDef,True,"[ineffective_defense, drives_under_stage, avoi...",0,Jim,399,4025,red_2,0,0,0,1,0,False,False,True


## Superapps Data Analysis

### Average Fouls per Match for Different Foul Types

In [35]:
output_df["Average In Bot Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.inBot"].mean()
output_df["Average Damage Bot Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.damageBot"].mean()
output_df["Average Tip Entangle Bot Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.tipEntangBot"].mean()
output_df["Average Pin Bot Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.pinBot"].mean()
output_df["Average Podium Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.podiumFoul"].mean()
output_df["Average Zone Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.zoneFoul"].mean()
output_df["Average Stage Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.stageFoul"].mean()
output_df["Average Over Exc Chute Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.overExtChute"].mean()

KeyError: 'Column not found: fouls.inBot'

### Average Total Fouls per Match

In [ ]:
output_df["Average Fouls per Match"] = None

for team in team_list:
    output_df.at[team, "Average Fouls per Match"] = sum([
        output_df.at[team, "Average In Bot Fouls per Match"]
        + output_df.at[team, "Average Damage Bot Fouls per Match"]
        + output_df.at[team, "Average Tip Entangle Bot Fouls per Match"]
        + output_df.at[team, "Average Pin Bot Fouls per Match"]
        + output_df.at[team, "Average Podium Fouls per Match"]
        + output_df.at[team, "Average Zone Fouls per Match"]
        + output_df.at[team, "Average Stage Fouls per Match"]
        + output_df.at[team, "Average Over Exc Chute Fouls per Match"]])

### Percent of Different Defense Types in Matches AND Main Defense Type

In [36]:
output_df["Percent of Matches with No Defense"] = None
output_df["Percent of Matches with Some Defense"] = None
output_df["Percent of Matches with Full Defense"] = None
output_df["Main Defense Type"] = None

for team in team_list:
    
    defense_type_list = superapps_df.loc[superapps_df["metadata.robotTeam"] == team, "defense"].tolist()
    
    no_defense_count = 0
    some_defense_count = 0
    full_defense_count = 0
    
    for defense_type in defense_type_list:
        if defense_type == "noDef":
            no_defense_count += 1
        elif defense_type == "someDef":
            some_defense_count += 1
        elif defense_type == "fullDef":
            full_defense_count += 1

    defense_total_count = sum([no_defense_count, some_defense_count, full_defense_count])
        
    output_df.at[team, "Percent of Matches with No Defense"] = no_defense_count / defense_total_count
    output_df.at[team, "Percent of Matches with Some Defense"] = some_defense_count / defense_total_count
    output_df.at[team, "Percent of Matches with Full Defense"] = full_defense_count / defense_total_count
    
    biggest_defense_count = max([no_defense_count, some_defense_count, full_defense_count])
    main_defense_type = ""

    if biggest_defense_count == no_defense_count:
        main_defense_type += "No Defense "
    if biggest_defense_count == some_defense_count:
        main_defense_type += "Some Defense "
    if biggest_defense_count == full_defense_count:
        main_defense_type += "Full Defense"
    
    output_df.at[team, "Main Defense Type"] = main_defense_type

## Comment Counts

In [37]:
exploded = superapps_df.explode('comments')
comment_counts = exploded.groupby('metadata.robotTeam')['comments'].apply(lambda x: [x.value_counts().to_dict()]).apply(lambda x: x[0])
# Yes that is jank but it wouldn't dictify nicely otherwise

output_df['Comments'] = comment_counts


## Additionals

# FINALIZE AND SEND DATA

## Send Data

In [38]:
output_df['teamNumber'] = output_df.index

In [39]:
output_df.to_json("../server/static/output_analysis.json", orient='records', default_handler=str)
output_df

,Auto Speaker Notes Near Avg,Auto Speaker Notes Mid Avg,Auto Speaker Notes Far Avg,Auto Speaker Notes Total Avg,Auto Speaker Notes Near Max,Auto Speaker Notes Mid Max,Auto Speaker Notes Far Max,Auto Speaker Notes Total Max,Auto Notes Amp Avg,Auto Percent of Missed Notes,...,Trap Notes Avg,Percent of Successful Climbs,Successful Climbs Total,Percent of Times Robot Left Starting Zone During Autos,Percent of Matches with No Defense,Percent of Matches with Some Defense,Percent of Matches with Full Defense,Main Defense Type,Comments,teamNumber
metadata.robotTeam,,,,,,,,,,,,,,,,,,,,,
11,1.947368,2.052632,2.421053,6.421053,4,4,4,10,2.631579,1.054054,...,0.526316,0.833333,10,0.578947,0.315789,0.368421,0.315789,Some Defense,"{'weak_build': 10, 'great_driving': 7, 'effect...",11
46,1.724138,2.068966,2.379310,6.172414,4,4,4,12,1.965517,1.26,...,0.482759,0.666667,10,0.620690,0.37931,0.310345,0.310345,No Defense,"{'avoids_under_stage': 13, 'ineffective_defens...",46
62,2.111111,1.740741,2.222222,6.074074,4,4,4,11,2.111111,0.789474,...,0.518519,0.714286,10,0.629630,0.296296,0.407407,0.296296,Some Defense,"{'effective_defense': 10, 'drives_under_stage'...",62
178,1.055556,2.666667,2.333333,6.055556,3,4,4,10,1.944444,2.263158,...,0.388889,0.692308,9,0.611111,0.388889,0.222222,0.388889,No Defense Full Defense,"{'good_driving': 6, 'clogging': 5, 'effective_...",178
428,1.800000,2.160000,1.680000,5.64,4,4,4,10,1.720000,1.311111,...,0.520000,0.8,12,0.560000,0.6,0.32,0.08,No Defense,"{'avoids_under_stage': 9, 'sturdy_build': 8, '...",428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9577,1.640000,2.400000,2.120000,6.16,4,4,4,10,2.160000,1.317073,...,0.480000,0.461538,6,0.480000,0.4,0.44,0.16,Some Defense,"{'weak_build': 12, 'sturdy_build': 9, 'great_d...",9577
9642,1.476190,1.666667,2.476190,5.619048,4,4,4,9,1.476190,1.290323,...,0.666667,0.916667,11,0.380952,0.285714,0.285714,0.428571,Full Defense,"{'avoids_under_stage': 11, 'effective_defense'...",9642
9702,1.714286,2.095238,1.476190,5.285714,4,4,4,10,2.333333,1.083333,...,0.571429,0.769231,10,0.523810,0.380952,0.380952,0.238095,No Defense Some Defense,"{'mid_defense': 9, 'good_driving': 7, 'effecti...",9702


In [40]:
client.close()